

# Building a Finder Chart with SDSS and Montage

<p style="color:#c00000;">In this notebook we will choose a region of the sky and dataset to mosaic, retrieve the SDSS archive data, reproject the images, search a catalog to be used as an overlay and finally build an output mosaic.  You are free to modify any of the mosaic parameters but beware that as you go larger all of the steps will take longer (possibly <b>much</b> longer).  If you do this for three different wavelenths, you can put them together in a full-color composite.</p>

As with many notebooks, this was derived from a longer script by breaking the processing up into sequential steps.
These steps (cells) have to be run one in sequence.  Wait for each cell to finish (watch for the step number in the brackets on the left to stop showing an asterisk) before starting the execution of next cell or run them all as a set.



## Setup

The Montage Python package is a mixture of pure Python and Python binary extension code.  It can be downloaded using <tt style="color: #c00000;">pip install MontagePy</tt>.  This notebook also assumes that Astropy and Astroquery have been installed.


In [7]:
!conda install conda-forge::astroquery

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - astroquery

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [6]:
import os
import sys
import shutil
import warnings
import sdss_archive
import datetime

from astropy.io import ascii

from astroquery.utils.tap.core import Tap

from MontagePy.main    import *
from MontagePy.archive import *

from IPython.display import Image

start = datetime.datetime.now()
print('Start: ' + str(start))


# These are the parameters defining the mosaic we want to make.

location  = "M51"
size      = 0.25
band      = "g"
workdir   = "M51g"

# Use user storage
# TODO: Test on SciServer to make sure folder is accessible
myUserName = Authentication.getKeystoneUserWithToken(Authentication.getToken()).userName
workdir = "/home/idies/workspace/Storage/" + myUserName + "/montage_temp/" + workdir + '/'

ModuleNotFoundError: No module named 'astroquery'


## Working Environment

Before we get to actually building the mosaic, we need to set up our working environment.  Given the volume of data possible, the Montage processing is file based and we need to set up some subdirectories to hold bits of it.  This will all be under an instance-specific directory specified above ("workdir").  It is best not to use directory names with embedded spaces.

In [ ]:
try:
    home
except:
    home = os.getcwd()

os.chdir(home)

print("Startup folder: " + home)


# Clean out any old copy of the work tree, then remake it
# and the set of the subdirectories we will need.

try:
    shutil.rmtree(workdir)
except:
    print("Can't delete work tree; may not exist or the device may be busy.", flush=True)

print("Work directory: " + workdir, flush=True)

try:
    os.makedirs(workdir)  
except:
    pass
    
os.chdir(workdir)

try:
    os.makedirs("raw")
except:
    pass

try:
    os.makedirs("projected") 
except:
    pass

try:
    os.makedirs("diffs")
except:
    pass 

try:
    os.makedirs("corrected")
except:
    pass

## Retrieving Data from an Archive

Now the first bit of Montage processing.  Montage uses standard FITS files throughout and FITS files have all the metadata describing the image (for us that mainly means pixel scale, projection type and center, rotation and so on).  To drive the processing we need a "FITS header" specification from the user, which we capture in a header "template" file that looks just like a FITS header though with newlines (FITS headers have fixed 80-character card images with no line breaks).  The mHdr routine is a utility that creates a simple FITS header template with limited control over all the above (<i>e.g.</i> the projection is always gnomonic (TAN)).  Other common options are to use a header extracted from some pre-existing FITS file (to create a matching mosaic) or to use mMakeHdr, which fits a bounding box around a set of images (usually the ones you are about to mosaic).

We also use the location and size to retrieve the data we want from a remote archive.  Montage provides an image metadata search service (using mSearch &mdash; a very fast R-Tree / memory-mapped utility &mdash; for most datasets).  This service returns URLs for all the images covering the region of interest, which are then downloaded.

There are many other ways to find images.  The International Virtual Astronomy Alliance (IVOA) has developed a couple of standards for querying metadata (Simple Image Access: SIAP and Table Access Protocol: TAP) which many data providers support.  Our service is focused on a few large uniform datasets (2MASS, DSS, SDSS, WISE).  Other datasets require more care.  For instance, simply downloading all pointed observations of a specific region for a non-survey instrument will include a wide range of integration times (and therefore noise levels) and the mosaicking should involve user-specified weighting of the images (which Montage supports but does not define).

In [ ]:
# Create the FITS header for the mosaic.
#os.chdir(workdir+'raw/')
rtn = mHdr(location, size, size, "region.hdr")

print("mHdr:             " + str(rtn), flush=True)


# Retrieve archive images covering the region then scan 
# the images for their coverage metadata.

rtn = sdss_archive.sdssDownload(band, location, size, workdir+"raw")

print("mArchiveDownload: " + str(rtn), flush=True)

rtn = mImgtbl(workdir+"raw", "rimages.tbl")

print("mImgtbl (raw):    " + str(rtn), flush=True)


## Reprojecting the Images

In the last step above, we generated a list of all the images (with projection metadata) that had been successfully downloaded.  Using this and header template from above, we can now reproject all the images to a common frame.

Montage supplies four different reprojection modules to fit different needs.  mProject is completely general and is flux conserving but this results in it being fairly slow.  For a subset of projections (specifically where both the source and destination are tangent-plane projections) we can use a custom plane-to-plane algorithm developed by the Spitzer Space Telescope).  While mProjectPP only supports a subset of cases, they are extremely common ones.  mProjectPP is also flux conserving.

For fast reprojection, we relax the flux conservation requirement. However, even though we call attention to this explicitly in the name of the module: mProjectQL (quick-look), the results are indistinguishable from the other algorithms for all the tests we have run.

The fourth reprojection module, mProjectCube, is specifically for three- and four-dimensional datacubes.

mProjExec is a wrapper around the three main reprojection routines that determines whether mProjectPP or mProject should be used for each image (unless overridden with mProjectQL as here).  There is one final twist:  FITS headers allow for distortion parameters.  While these were introduced to deal with instrumental distortions, we can often use them to mimic an arbitrary projection over a small region with a distorted gnomonic projection.  This allows us to use mProjectPP over a wider range of cases and still have flux conservation with increased speed.

In [ ]:
# Reproject the original images to the  frame of the 
# output FITS header we created
#os.chdir(workdir+'raw/')

rtn = mProjExec("raw", "rimages.tbl", "region.hdr", projdir="projected", quickMode=True)

print("mProjExec:           " + str(rtn), flush=True)

mImgtbl("projected", "pimages.tbl")

print("mImgtbl (projected): " + str(rtn), flush=True)


## Coadding for a Mosaic

Now that we have a set of image all reprojected to a common frame, we can coadd them into a mosaic.  

In [ ]:
# Coadd the projected images without backgound correction.

rtn = mAdd("projected", "pimages.tbl", "region.hdr", "uncorrected.fits")

print("mAdd:    " + str(rtn), flush=True)


## Searching a Catalog 



In [ ]:
# There are Table Access Protocol services at CDS, IRSA, GAIA, NED etc.

warnings.filterwarnings('ignore')
   
service = Tap(url="https://irsa.ipac.caltech.edu/TAP")

job = service.launch_job_async("""SELECT ra, dec, j_m, h_m, k_m FROM fp_psc 
                                  WHERE CONTAINS(POINT('ICRS',ra, dec), CIRCLE('ICRS',202.48417,47.23056,0.4))=1""",
                                  verbose=False, background=True)
res = job.get_results()

ascii.write(res, output='fp_2mass.tbl', format='ipac')

## View the Image

FITS files are binary data structures. To see the image we need to render to a JPEG or PNG form.  This involves choosing a stretch, color table (if it is a single image as here) and so on.  Montage provides a general visualization tool (mViewer) which can process a single image or multiple images for full color.  It supports overlays of various sorts.  One of its most useful features is a custom stretch algorithm which is based on gaussian-transformed histogram equalization, optionally with an extra logarithmic transform for really bright excursions.  A large fraction of astronomical images share the general characteristics of having a lot of pixels with something like a gaussian distribution at a low flux level (either background noise of low-level sky structure) coupled with a long histogram tail of very bright point-like sources.  If we apply our algorithm to this, stretching from the -2 or -1 "sigma" value of the low-level distribution to the image brightness maximum we usually get a good balance of seeing the low-level structure while still seeing the structure and brightness variations of the bright sources.

mViewer specifications can become quite lengthy so the module provides three entry mechanisms.  The most terse (used here) is a "parameter string" based on the command-line arguments of the original stand-alone C program.  For more complicated descriptions the user can define a JSON string or JSON file.  See the <a style="text-decoration: none; color: #c00000" href="mViewer.ipynb"> Sky Visualization </a> notebook example.

We use the built-in IPython.display utility to show the resultant image, which shrinks it to fit.  There are several other tools you can use instead.

In [ ]:
# Make a PNG rendering of the data and display it.

rtn = mViewer('''-ct 1 
                 -color blue
                 -grid equ j2000
                 -color red 
                 -symbol 1.0 circle 
                 -scalecol j_m 16.0 mag 
                 -catalog fp_2mass.tbl 
                 -gray uncorrected.fits -2s max gaussian-log -out uncorrected.png''',
               "", mode=2)

print("mViewer: " + str(rtn), flush=True)

Image(filename='uncorrected.png')

In [ ]:
end = datetime.datetime.now()
print('End: ' + str(start))

runtime = end - start

print('\nTotal time: ' + str(runtime))